<center><h1>Projekt</h1></center>
<center><h1>Analiza wpisów profilu facebook'owego portalu Nowy Ład z wykorzystaniem algorytmu Word2Vec.</h1></center>
<h3>Autor: Krzysztof Jarek</h3>

<h2><br>Wstęp teoretyczny.</h2>

Word2Vec jest to algorytm, czy bardziej precyzyjnie grupa metod, wykorzystywany w przetwarzaniu języka naturalnego (NLP). Zaliczny jest do szerszej grupy technik tzw. osadzania słów, inaczej opartych o <i>word embedding</i>, podejścia polegającego na reprezentowaniu, zapisywaniu słów za pomocą wektorów, które następnie jako struktury matematyczne mogą być wykorzystywane dla celów dalszej analizy, obliczeń [1].<br><br>
Samo wspomniane <i>word embedding</i> w swojej idei opiera się o założenie przekształcenia tekstu na liczby (ustrukturyzowane zgrupowania liczb, np. wektory), przy czym dla tego samego tekstu mogą istnieć różne reprezentacje liczbowe. Wybór tego rozwiązania wynika z faktu, że wiele algorytmów uczenia maszynowego i generalnie wszystkie głębokiego uczenia nie są w stanie przetwarzać ciągów znaków ani zwykłego surowego tekstu. Z tego powodu potrzebują reprezentacji liczbowych jako danych wejściowych do wykonania dowolnego rodzaju przetwarzania (klasyfikacji, regresji). Można wyróżnić dwa rodzaje <i>word embedding</i>: oparte na częstotliwości (np. TF-IDF) i oparte na predykcji [2].<br><br>
Wspomniane reprezentacje wektorowe, lub te oparte na częstotliwości, w generowanych za pomocą algorytmów modelach bazują na macierzach współwystępowania (<i>co-occurrence matrix</i>), które są sposobami przedstawiania jak często konkretne słowa współwystępują. Dwoma najpopularniejszymi typami macierzy są: macierze terminów-dokumentów i macierze terminów-terminów. Padające określenie współwystępowania jest tu rozumiane w ten sposób, że w przypadku danego korpusu słów przy pojawianiu się danej pary słów w zdefiniowanym oknie kontekstowym odpowiada liczba wyrażajaca w ten sposób matematyczne podobieństwo (<i>similarity</i>) tych słów [2].

<center><img src="images/word_embedding.png" width=650 height=650 /></center>
<center>Rys. 1 Przedstawienie idei okna kontesktowego dla wektorowego <i>word embedding</i> [3].</center>

W ramach domeny NLP jako metryka dla mierzenia wspomnianego podobieństwa (<i>similarity</i>) słów reprezentowanych przez wektory z przyczyn praktycznych przyjęła się metryka cosinusowa. Jej wzór prezentuje się następująco:<br>
<center><img src="images/cosine_metric.png" width=450 height=450 /></center>
<center>gdzie: <b>v, w</b> to wektory [4].</center>

Wracając do samej metody Word2Vec- jej wykorzystanie w celu stworzenia modelu służącego do rekonstrukcji językowych kontekstów słów opiera się o to, że przygotowany korpus słów jest wykorzytywany w ramach procesu uczenia dwuwarstwowych sieci neuronowe, który to proces ma ostatecznie doprowadzić do wygenerowania przestrzeni wektorowej w jakiej każdemu słowu ze zbioru wejściowego przypisywany jest odpowiedni wektor (w tak uzyskanej przestrzeni wektorowej). Jeśli chodzi o metodę to istotne jest to, iż pozwala ona na to, że słowa powiązane ze sobą kontekstem mają wektory znajdujące się blisko siebie w przestrzeni wyjściowej [5].

<center><img src="images/example.png" width=550 height=550 /></center>
<center>Rys. 2 Przykład przestrzeni wyjściowej dla Word2Vec [6].</center>

Ze względu na architekturę wyróżnia się dwa główne podejścia Word2Vec, gdzie decydującą cechą jest charakter dokonywania rekonstrukcji językowych kontekstów słów:
<ul>
    <li>CBOW (<i>continuous bag-of-words</i>)- jakie opiera się o założenie, że na podstawie słów tworzących kontekst dokonuje się predykcji słowa, które w takim kontekście powinno się znajdować,</li>
    <li>Skip-gram- odwrotnie, dla jakiego na podstawie słowa dokonuje się predykcji kontekstu składającego się z otaczających słów [5].</li>
</ul>

<center><img src="images/architectures.png" width=600 height=600 /></center>
<center>Rys. 3 Zasada działania architektur CBOW i Skip-gram [5].</center>

Jeśli chodzi o funkcję uczenia (funkcję celu) dla modeli Word2Vec to można wymienić jej specyficzne stosowane odmiany. Pierwotnie stosowana była po prostu funkcja <i>softmax</i> przyjmująca wektory reprezentujące słowa, ale z racji na obciążenie obliczeniowe została zastąpiona przez następujące:
<ul>
    <li>hierarchiczny softmax- gdzie liczba obliczeń jest redukowana poprzez wykorzystanie drzewa binarnego (np. drzewo Huffman'a) za sprawą użycia jakiego tylko część węzłów w sieci neuronowej musi być wykorzystana dla danego etapu obliczeń ( $log_{2}$<i>liczba węzłów</i> ),</li>
    <li>negatywne próbkowanie- rozwiązuje problem maksymalizacji prawdopodobieństwa poprzez minimalizację prawdopodobieństwa (logarytmicznego) próbkowanych wystąpień negatywnych [7].</li>
</ul>

<h2><br><br><br>Część praktyczna.</h2>

W tej części projektu przeszedłem do praktycznego wykorzystania wcześniej omówionej metody. Jako zbiór uczący dla przeprowadzenia analizy przygotowałem korpus składający się z 95 wpisów z serwisu facebook profilu portalu Nowy Ład. Profil ten wybrałem z racji na sam charakter wpisów: generalnie nie lakonicznych (jak np. profile Christianitas, OkoPress) czy zawierających tekst jako taki bez licznych linków, treści promocyjnych.<br><br>
Do samej analizy wykorzystałem technologię języka python3 i narzędzia dostarczone przez pakiety takie jak <i>gensim</i> (Word2Vec), <i>nltk</i>, <i>re</i>, <i>pandas</i>. Pracę zacząłem od działań takich jak wczytanie przygotowanych danych, poglądowe ich wyświetlenie i wstępne przetworzenie do postaci zdatnej do poddania ich procesowi uczenia. Wszytko wymienione zostało umieszczone przeze mnie poniżej:

In [1]:
import re
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/fb_posts.csv', delimiter=';')

In [3]:
print('Liczba wykorzystanych wpisów z facebook\'a:', df.shape[0])

Liczba wykorzystanych wpisów z facebook'a: 95


In [4]:
df.head()

,Date,Post
0,13-07-2022,Erę rządów Abe można ocenić pozytywnie głównie...
1,13-07-2022,Pierwsze dni lipca upłynęły w Uzbekistanie pod...
2,13-07-2022,Od początku działalności naszej młodej inicjat...
3,13-07-2022,Zapraszamy do dołączenia do grupy sympatyków p...
4,12-07-2022,Coraz konsekwentniej realizowana przez ustawod...


In [5]:
df.tail()

,Date,Post
90,14-06-2022,Zagrożenia ekologiczne są jednym z kluczowych ...
91,14-06-2022,Czy rosyjskie elity trwale zwrócą się przeciwk...
92,13-06-2022,Wobec największego po 1989 r kryzysu w naszym ...
93,13-06-2022,Trwająca od ponad 100 dni wojna na Ukrainie po...
94,12-06-2022,Jakie są historyczne i ideowe źródła tożsamośc...


In [6]:
all_posts = df.Post.tolist()

In [7]:
corpora = []
[corpora.extend(sent_tokenize( re.sub(r'[„”"@’]', '', text) )) for text in all_posts];

In [8]:
print('Liczba zdań składających się na korpus:', len(corpora), '.')

Liczba zdań składających się na korpus: 408 .


In [9]:
sentences = [word_tokenize(sentence) for sentence in corpora]

In [10]:
for i in range(len(sentences)):
    sentences[i] = [word.lower() for word in sentences[i] if re.match('^[a-zA-Z]+', word)]

Tekst jako taki poddałęm procesowi oczyszczenia ze znaków, które nie wnosiłyby zbyt wiele dla przeprowadzanej analizy, a mogłyby np. obniżyć rezultaty przeprowadzanego procesu uczenia (takich jak cydzysłów itd.). Dalej, korpus podzieliłem, wyodrębniając poszczególne zdania. Mając już dostępne same zdania, przetransformowałem je do postaci list składających się z ciągami znaków:

In [11]:
for i in range(5):
    print(sentences[i])

['erę', 'rządów', 'abe', 'można', 'ocenić', 'pozytywnie', 'głównie', 'z', 'powodu', 'swoistego', 'odrodzenia', 'narodowego', 'japonii']
['państwo', 'ponownie', 'rozbudziło', 'wśród', 'społeczeństwa', 'uczucia', 'patriotyczne', 'między', 'innymi', 'poprzez', 'prowadzenie', 'odpowiedniej', 'polityki', 'edukacyjnej', 'wśród', 'najmłodszej', 'części', 'populacji']
['imponujący', 'jest', 'także', 'rozwój', 'japońskich', 'sił', 'samoobrony', 'które', 'także', 'pod', 'obecnymi', 'rządami', 'mogą', 'liczyć', 'na', 'coraz', 'większe', 'fundusze']
['z', 'drugiej', 'strony', 'zamordowany', 'były', 'premier', 'nie', 'poradził', 'sobie', 'na', 'polu', 'gospodarczym', 'nie', 'odwracając', 'negatywnych', 'trendów', 'ostatnich', 'trzech', 'straconych', 'dekad']
['i', 'tak', 'spuścizna', 'abe', 'pozostanie', 'jednak', 'na', 'bardzo', 'długo', 'nie', 'tylko', 'z', 'powodu', 'jego', 'tragicznej']


In [12]:
from gensim.models import Word2Vec
from multiprocessing import Pool

Proces uczenia przeprowadziłem bezpośrednio uruchamijąc go wraz z utworzeniem obiektu dostarczonej klasy Word2Vec (domyślna działanie konstruktora). Wyjaśnienie parametrów konstruktora klasy:
<ul>
    <li>sentences - dostarczony zbiór danych treningowych,</li>
    <li>sg - jeśli 1 to realizowany jest Skip-gram (inaczej CBOW),</li>
    <li>window - liczba słów tworzących okno kontekstowe,</li>
    <li>min_count - minimalna liczba zliczeń dla słowa tak by zostało dołączone do wyjściowego słownika,</li>
    <li>epochs - liczba epok uczenia sieci,</li>
    <li>workers - liczba wątków na jakie zostanie rozłożone wykonanie procesu.</li>
</ul>

In [13]:
model = Word2Vec(
    sentences = sentences, 
    sg = 1, # skip-gram
    window = 5, 
    min_count = 1, 
    epochs = 8, 
    workers = Pool()._processes
)

In [14]:
model.save('word2vec.model')

In [15]:
model = Word2Vec.load('word2vec.model')

Następnie przeszedłem do samej analizy. Pakiet <i>gensim</i> dostarcza metodę obiektu typu Word2Vec <i>most_similar()</i>, która zwraca <i>n</i> słów, które mają najwyższy współczynnik podobieństwa względem słowa podanego na wejściu. Poniżej zaprezentowałem szereg wywołań tej metody z wybranymi słowami, jakie zwróciły następujące rezultaty:

In [16]:
model.wv.most_similar('polska')

[('i', 0.9666223526000977),
 ('na', 0.9662830829620361),
 ('do', 0.9661742448806763),
 ('dla', 0.965888500213623),
 ('o', 0.9655967354774475),
 ('polski', 0.9655476808547974),
 ('przez', 0.965462863445282),
 ('ii', 0.965330183506012),
 ('się', 0.9652944803237915),
 ('z', 0.9650025963783264)]

In [17]:
model.wv.most_similar('turcja')

[('polski', 0.9506455659866333),
 ('dr', 0.9505966901779175),
 ('ukrainy', 0.9479331970214844),
 ('jak', 0.9475066661834717),
 ('i', 0.9464317560195923),
 ('się', 0.946362316608429),
 ('dla', 0.9463019967079163),
 ('z', 0.9461473226547241),
 ('jako', 0.9461274743080139),
 ('to', 0.9461025595664978)]

In [18]:
model.wv.most_similar('afryki')

[('utrata', 0.6542149186134338),
 ('zachodnią', 0.6423894166946411),
 ('wysokiej', 0.6416317224502563),
 ('wojnę', 0.6303442120552063),
 ('kontynuują', 0.6283208131790161),
 ('uam', 0.6236923336982727),
 ('zwiększyć', 0.6207494139671326),
 ('zrobić', 0.6182249188423157),
 ('biblioteki', 0.6180039048194885),
 ('aby', 0.6174520254135132)]

In [19]:
model.wv.most_similar('ukraina')

[('dla', 0.9743965864181519),
 ('nie', 0.9740657210350037),
 ('do', 0.9736761450767517),
 ('tylko', 0.9736183285713196),
 ('w', 0.9734273552894592),
 ('a', 0.97282475233078),
 ('o', 0.972655713558197),
 ('to', 0.9726532101631165),
 ('być', 0.9725211262702942),
 ('za', 0.9724376797676086)]

In [20]:
model.wv.most_similar('usa')

[('to', 0.9816751480102539),
 ('i', 0.9816627502441406),
 ('jest', 0.9814104437828064),
 ('w', 0.9808385372161865),
 ('czy', 0.9807396531105042),
 ('się', 0.9802049398422241),
 ('od', 0.9801687002182007),
 ('o', 0.9801136255264282),
 ('co', 0.980099081993103),
 ('polski', 0.9800916314125061)]

In [21]:
model.wv.most_similar('premier')

[('tylko', 0.9455297589302063),
 ('ze', 0.9452150464057922),
 ('jest', 0.9439623951911926),
 ('można', 0.9426714777946472),
 ('są', 0.9424187541007996),
 ('a', 0.942260205745697),
 ('najbardziej', 0.9418258666992188),
 ('o', 0.940778374671936),
 ('od', 0.9407326579093933),
 ('po', 0.9407224655151367)]

In [22]:
model.wv.most_similar('warszawa')

[('albo', 0.7203791737556458),
 ('stwarzają', 0.7122724652290344),
 ('samodzielną', 0.7062618732452393),
 ('kierunku', 0.7062522768974304),
 ('kity', 0.704880952835083),
 ('dołączenia', 0.7046069502830505),
 ('większość', 0.7036066055297852),
 ('wynik', 0.7019095420837402),
 ('ukrainie', 0.7018398642539978),
 ('ilości', 0.701668381690979)]

Dla przyjrzenia się ściślej problemowi preferowanej w NLP metryki cosinusowej, przygotowałem specjalną implementację dla przebadania tego jakie wartości mogą zostać zwrócone dla przypadków wybranych par słów. Implementację wraz z przykładami zamieściłęm poniżej:

In [23]:
from scipy import spatial

In [24]:
def cosine_similarity(wv0, wv1):
    return 1 - spatial.distance.cosine(wv0, wv1)

In [25]:
wv0 = model.wv['warszawa']
wv1 = model.wv['miasto']
cosine_similarity(wv0, wv1)

0.5165625810623169

In [26]:
wv0 = model.wv['politycy']
wv1 = model.wv['polityki']
cosine_similarity(wv0, wv1)

0.9111180901527405

In [27]:
wv0 = model.wv['polska']
wv1 = model.wv['ukraina']
cosine_similarity(wv0, wv1)

0.9476596117019653

In [28]:
wv0 = model.wv['ukraina']
wv1 = model.wv['francja']
cosine_similarity(wv0, wv1)

0.9322970509529114

In [29]:
wv0 = model.wv['ukraina']
wv1 = model.wv['rosji']
cosine_similarity(wv0, wv1)

0.9177106022834778

In [30]:
wv0 = model.wv['ukraina']
wv1 = model.wv['turcja']
cosine_similarity(wv0, wv1)

0.9161891341209412

Na koniec ponownie podjąłem się wykorzystania metody <i>most_similar()</i>- tym razem wykorzystując możliwość pooperowania na wbudowanych parametrach <i>positive</i> i <i>negative</i>, które to pozwalają nie tylko na pozyskiwanie probabilistycznych wyników zwracanych przez model dla wybranych słów czy zbiorów słów, ale co więcej pozwalają na przeprowadzanie operacji algebraicznych. Operacje te faktycznie są przeprowadzane jako operacje na wektorach reprezentujących słowa- ta oto idea możliwości przeprowadzania operacji algebraicznych była celem, który środowisko zajmujące się NLP bardzo chciało osiągnąć. Przykładowe operacje zamieściłem poniżej:

In [31]:
model.wv.most_similar(positive=['warszawa','turcja'],negative=['polska'],topn=2)

[('albo', 0.6754295825958252), ('konserwatywny', 0.6609734296798706)]

In [32]:
model.wv.most_similar(positive=['francja','bez','kitu'],topn=1)

[('polacy', 0.9354678988456726)]

In [33]:
model.wv.most_similar(positive=['facebooka','polityka'],negative=['google'],topn=3)

[('kryzysu', 0.6659268140792847),
 ('stary', 0.6442970037460327),
 ('system', 0.6421110033988953)]

In [34]:
model.wv.most_similar(positive=['henry','kissinger','powiedział'],topn=2)

[('wobec', 0.9582085609436035), ('polski', 0.9562836289405823)]

<h2>Wnioski:</h2>
<ul>
    <li>Word2Vec jest metodą pozwalającą dokonywać w sposób bardzo wyrafinowany rekonstruowania językowych kontekstów związanych z konkretnymi słowami, zgrupowaniami słów. Co więcej pozwala ona zachowywać liniowe zależności występujące między słowami (reprezentowanymi przez wektory) w ich oryginalnych kontekstach lingwistycznych.</li>
    <li>Word2Vec pozwala na posługiwanie się obiektami wyjściowej przestrzeni wektorowej w sposób bardziej zaawansowany, dostarczając możliwości wykorzystywania operacji algebraicznych <i>de facto</i> odnoszących się bezpośrednio do samych słów. Co jest dużym rozwinięciem analitycznym i technologicznym dla domeny NLP.</li>
    <li>By rezultaty zwracane prez Word2Vec były satysfakcjonujące z perspektywy przeprowadzania procesu uczenia, wskazane jest by przygotowywać odpowiednio duże zbiory tekstowe (przygotowany dla badania mógł być niedostatecznie duży).</li>
    <li>Na podstawie samej przeprowadzonej analizy można zauważyć, że w przebadanych kontekstach słów leksy słów "polski" i "Ukraina" często pojawiały się na listach słów podobnych do badanych. Poza tym dominowały zdecydowanie spójniki.</li>
</ul>

<h2>Przypisy:</h2>

[1] D. Jurafsky, H. J. Martin, "Vector Semantics and Embeddings", s. 17-18, [w:] <i>Speech and language processing : an introduction to natural language processing, computational linguistics, and speech recognition.</i>, 2021, [na:] https://web.stanford.edu/~jurafsky/slp3/, [dostęp:] 17.07.2022.<br>
[2] <i>Ibidem</i>, s. 5-10.<br>
[3] J. Ramkissoon, "First Steps with Word Embeddings", 2019, [na:] https://jramkiss.github.io/2019/08/21/word-embeddings/, [dostęp:] 17.07.2022.<br>
[4] D. Jurafsky, H. J. Martin, "Vector Semantics...", s. 10-11.<br>
[5] T. Mikolov (red.), "Efficient Estimation of Word Representations in Vector Space", 2013, s. 4-5.<br>
[6] T. Mikolov, W. Yih, G. Zweig, "Linguistic Regularities in Continuous Space Word Representations", 2013, s. 4.<br>
[7] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, J. Dean, "Distributed representations of words and phrases and their compositionality", 2013, s. 2-4.<br>